## Read Config

In [1]:
import json

# 读取 JSON 文件
with open("config.json", "r", encoding="utf-8") as f:
    config = json.load(f)


## Text Splite

### 1. LangChain: Splitting long texts while preserving semantic integrity.

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_text(file_path, chunk_size=2000, chunk_overlap=200):
    """
    Reads a novel text file and splits it into smaller chunks for processing.

    Parameters:
        file_path (str): The path to the novel text file.
        chunk_size (int): The maximum number of characters per chunk. Default is 2000.
        chunk_overlap (int): The number of overlapping characters between chunks to preserve context. Default is 200.

    Returns:
        list: A list of text chunks.
    """
    # Read the novel text file
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    # Use recursive text splitting to maintain coherence
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_text(text)

    return chunks


In [6]:
# 访问配置数据
novel_pth = config["project_paths"]["data_dir"]
chunk_size=config["text_splitter"]["chunk_size"]
chunk_overlap=config["text_splitter"]["chunk_overlap"]
# 读取小说文本并拆分成片段
novel_chunks = load_and_split_text(novel_pth, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
print("chunk_size:", chunk_size)
print("chunk_overlap:", chunk_overlap)

chunk_size: 1000
chunk_overlap: 100


In [7]:
# ✅ Verify the output
print(type(novel_chunks))  # Check the data type of the output
print(f"Total {len(novel_chunks)} scene segments extracted")  # Display the number of text chunks
print(novel_chunks[0])  # Preview the first chunk


<class 'list'>
Total 101 scene segments extracted
*** START OF THE PROJECT GUTENBERG EBOOK THE CALL OF CTHULHU ***
The CALL of CTHULHU

By H.P. LOVECRAFT

[Transcriber's Note: This etext was produced from
Weird Tales, February 1928.
Extensive research did not uncover any evidence that
the U.S. copyright on this publication was renewed.]


    "Of such great powers or beings there may be conceivably a
    survival ... a survival of a hugely remote period when ...
    consciousness was manifested, perhaps, in shapes and forms long
    since withdrawn before the tide of advancing humanity ... forms
    of which poetry and legend alone have caught a flying memory
    and called them gods, monsters, mythical beings of all sorts
    and kinds...."

                                                --_Algernon Blackwood._


[Illustration: "The ring of worshipers moved in endless bacchanale
between the ring of bodies and the ring of fire."][1]

[Footnote 1: Found among the papers of the late Fra

### 2. Designing PromptTemplate for scene segmentation.

In [ ]:

from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["text"],
    template="""
You are a professional scriptwriter. Analyze the following novel text and divide it into multiple scenes.
Each scene should include:
- **Scene ID**
- **Scene Summary** (a brief description of what happens)
- **Main Characters**
- **Main Location**
- **Key Events**
- **Scene Transition Reason** (Why is this a new scene?)
- **Original Text** (The original text corresponding to this scene)

**Always return a strict JSON format** with **no extra text or explanations**, only pure JSON.
Return the output in **JSON format array**, following this example:
"scenes":[
    {{
        "scene_id": 1,
        "summary": "The protagonist finds a mysterious letter at home.",
        "characters": ["Protagonist"],
        "location": "Protagonist's house",
        "events": ["Finds the letter", "Reads the content"],
        "atmosphere": "Mysterious",
        "transition_reason": "A new event begins",
        "original_text": "He entered his home, only to find a dusty envelope on the table."
    }},
    {{
        "scene_id": 2,
        "summary": "The protagonist visits the mysterious location.",
        "characters": ["Protagonist", "Antagonist"],
        "location": "Mysterious forest",
        "events": ["Meets the antagonist", "Fights the antagonist"],
        "atmosphere": "Tense",
        "transition_reason": "The protagonist arrives at the location",
        "original_text": "He entered the forest, where he met the antagonist."
    }}
    ...
]

Here is the novel text:
{text}
"""
)

### 3. LLM Model init

In [37]:
import openai
openai.api_key = config["KEY"]["OPENAI_API_KEY"]

In [ ]:
import openai
import json

class Chatbot:
    def __init__(self, system_prompt):
        """
        Initializes the chatbot with a system prompt.

        Parameters:
            system_prompt (str): A predefined instruction that sets the chatbot's behavior.
        """
        self.system_prompt = system_prompt

    def generate_response(self, prompt):
        """
        Generates a response from OpenAI's GPT-4 Turbo model.

        Parameters:
            prompt (str): The user input message to which the chatbot responds.

        Returns:
            str: The generated response as a JSON-formatted string.
        """
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",  # Uses the GPT-4 Turbo model for optimized performance
            messages=[
                {"role": "system", "content": self.system_prompt},  # Defines system-level behavior
                {"role": "user", "content": prompt}  # User input prompt
            ],
            temperature=0.5,  # Lowers randomness to ensure structured and stable responses
            top_p=0.9,  # Prevents extreme or highly unlikely outputs
            n=1,  # Generates only one response
            response_format={"type": "json_object"},  # Forces GPT to return a JSON object
            presence_penalty=0.2,  # Slightly encourages new content in responses
            frequency_penalty=0.2,  # Slightly reduces repetitive phrases
            stop=["\n\n"]  # Stops response at the end of a paragraph
        )

        try:
            # Convert the output from JSON string format to a Python dictionary
            # return json.loads(response.choices[0].message["content"])
            return response.choices[0].message["content"]
        except json.JSONDecodeError as e:
            print(f"JSON ERROR: {e}")  # Prints error message if JSON decoding fails
            return None


In [120]:
# 初始化 Chatbot 并测试解析小说
chatbot = Chatbot(system_prompt="You are a screenplay expert. Return a structured JSON array.")

In [ ]:
novel_text = """My knowledge of the thing began in the winter of 1926-27 with the death
of my grand-uncle, George Gammell Angell, Professor Emeritus of Semitic
languages in Brown University, Providence, Rhode Island. Professor
Angell was widely known as an authority on ancient inscriptions, and
had frequently been resorted to by the heads of prominent museums; so
that his passing at the age of ninety-two may be recalled by many.
Locally, interest was intensified by the obscurity of the cause of
death. The professor had been stricken whilst returning from the
Newport boat; falling suddenly, as witnesses said, after having been
jostled by a nautical-looking negro who had come from one of the queer
dark courts on the precipitous hillside which formed a short cut from
the waterfront to the deceased's home in Williams Street. Physicians
were unable to find any visible disorder, but concluded after perplexed
debate that some obscure lesion of the heart, induced by the brisk
ascent of so steep a hill by so elderly a man, was responsible for
the end. At the time I saw no reason to dissent from this dictum, but
latterly I am inclined to wonder--and more than wonder."""
# ✅ Generate the prompt required for GPT
prompt = prompt_template.format(text=novel_text)

# ✅ Get the JSON data generated by GPT after processing the prompt
scene_data = chatbot.generate_response(prompt)

# ✅ Output the parsed result
print(scene_data)

# Uncomment the line below to pretty-print the JSON response
# print(json.dumps(scene_data, indent=4, ensure_ascii=False))

# print(json.dumps(scene_data, indent=4, ensure_ascii=False))

{
    "scenes": [
        {
            "scene_id": 1,
            "summary": "The narrative begins with the death of the protagonist's grand-uncle, Professor George Gammell Angell, under mysterious circumstances.",
            "characters": ["George Gammell Angell", "Protagonist"],
            "location": "Providence, Rhode Island",
            "events": ["Death of George Gammell Angell", "Mysterious circumstances surrounding the death"],
            "transition_reason": "Introduction to the main plot and backstory.",
            "original_text": "My knowledge of the thing began in the winter of 1926-27 with the death of my grand-uncle, George Gammell Angell, Professor Emeritus of Semitic languages in Brown University, Providence, Rhode Island. Professor Angell was widely known as an authority on ancient inscriptions, and had frequently been resorted to by the heads of prominent museums; so that his passing at the age of ninety-two may be recalled by many. Locally, interest was intens

In [128]:
# 解析 JSON 数据
scene_dict = json.loads(scene_data)
scenes_only = scene_dict.get("scenes", [])
print(f"共解析出 {len(scenes_only)} 个场景")
print(scenes_only[0])

共解析出 4 个场景
{'scene_id': 1, 'summary': "The narrative begins with the death of the protagonist's grand-uncle, Professor George Gammell Angell, under mysterious circumstances.", 'characters': ['George Gammell Angell', 'Protagonist'], 'location': 'Providence, Rhode Island', 'events': ['Death of George Gammell Angell', 'Mysterious circumstances surrounding the death'], 'transition_reason': 'Introduction to the main plot and backstory.', 'original_text': 'My knowledge of the thing began in the winter of 1926-27 with the death of my grand-uncle, George Gammell Angell, Professor Emeritus of Semitic languages in Brown University, Providence, Rhode Island. Professor Angell was widely known as an authority on ancient inscriptions, and had frequently been resorted to by the heads of prominent museums; so that his passing at the age of ninety-two may be recalled by many. Locally, interest was intensified by the obscurity of the cause of death.'}


In [131]:
import os
# JSON 文件路径
file_path = config["project_paths"]["scripts_dir"]
# 获取所在的目录路径
dir_path = os.path.dirname(file_path)

# **1. 如果目录不存在，则创建**
if not os.path.exists(dir_path):
    os.makedirs(dir_path)  # 创建所有需要的目录
# **2. 如果 JSON 文件不存在，则创建空文件**
if not os.path.exists(file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(scenes_only, f, ensure_ascii=False, indent=4)  # 预留一个空列表

## 使用Dramatron创建剧本

## 使用GraphRAG解析长文本小说

### 2.结合GPT+GraphRGA产生整体小说的视觉风格
### 3.利用streamlit来填写prompt，默认值是上一步返回的结果

### 4. 为主要角色创建人物画像，确保前后文的角色形象统一

## **Generate Illustrated Images**

In [ ]:
from langchain.prompts import PromptTemplate
import json

# ✅ Load the Visual_Style configuration file
with open("config.json", "r", encoding="utf-8") as f:
    config = json.load(f)

visual_style = config["Visual_Style"]  # Retrieve the Lovecraftian horror style settings

# ✅ Load the scenes_1.json file
with open("scripts/The_Call_of_Cthulhu/scenes_1.json", "r", encoding="utf-8") as f:
    scene_list = json.load(f)  # Load all scene details

# ✅ Generate image prompts
prompts = []

for scene in scene_list:  
    # Extract scene details
    scene_desc = scene["summary"]
    location = scene["location"]
    characters = ", ".join(scene["characters"])  # Convert character list to a string
    events = ", ".join(scene["events"])  # Convert event list to a string
    
    # ✅ Construct the final prompt using visual style settings
    prompt = (
        f"A {visual_style['mood']} scene set in {visual_style['time_period']}. "
        f"The art style is {visual_style['art_style']}, using {visual_style['color_palette']} colors. "
        f"The environment is {visual_style['details']['environment']} under {visual_style['details']['weather']}. "
        f"The setting is {location}, featuring {characters}. "
        f"Key events happening in this scene: {events}. "
        f"The scene is illuminated by {visual_style['details']['lighting']}. "
        f"Scene description: {scene_desc}."
    )
    
    prompts.append(prompt)

# ✅ Output all generated prompts
for i, p in enumerate(prompts):
    print(f"Prompt {i + 1}: {p}\n")


Prompt 1: A Eldritch, Uncanny, Cosmic Horror scene set in 1920s Gothic Horror. The art style is Dark Gothic, Lovecraftian Horror, using Dark, Muted Tones, Sepia, Greenish Black colors. The environment is Foggy coastal town, ancient cyclopean ruins, deep-sea abyss under Stormy night, fog-covered city, eerie full moon. The setting is Providence, Rhode Island, featuring George Gammell Angell, Protagonist. Key events happening in this scene: Death of George Gammell Angell, Mysterious circumstances surrounding the death. The scene is illuminated by Dim gas lamps, eerie green glow, unnatural shadows. Scene description: The narrative begins with the death of the protagonist's grand-uncle, Professor George Gammell Angell, under mysterious circumstances..

Prompt 2: A Eldritch, Uncanny, Cosmic Horror scene set in 1920s Gothic Horror. The art style is Dark Gothic, Lovecraftian Horror, using Dark, Muted Tones, Sepia, Greenish Black colors. The environment is Foggy coastal town, ancient cyclopean 

In [ ]:
import requests
import time
import os  # Used to create directories
import openai

# **📂 Directory to save generated images**
save_dir = "images/The_Call_of_Cthulhu"

# **📌 Check and create directory if it does not exist**
os.makedirs(save_dir, exist_ok=True)

# **🎨 Loop through prompts to generate images**
for i, prompt in enumerate(prompts):
    try:
        print(f"Generating image {i + 1} / {len(prompts)}: {prompt}")

        # **🖼️ Generate the image using DALL·E 3**
        response = openai.Image.create(
            prompt=prompt,
            model="dall-e-3",  # Uses the DALL·E 3 model
            n=1,  # Generates 1 image per request
            size="1024x1024"  # Image resolution
        )

        # **🔗 Retrieve the image URL from the response**
        image_url = response["data"][0]["url"]
        print(f"✅ Image {i + 1} generated successfully, URL: {image_url}")

        # **📥 Download the image**
        img_data = requests.get(image_url).content
        file_name = os.path.join(save_dir, f"generated_image_{i + 1}.png")
        with open(file_name, "wb") as img_file:
            img_file.write(img_data)

        print(f"📂 Image saved as {file_name}\n")

        # **⏳ Delay to avoid API rate limits**
        time.sleep(2)

    except Exception as e:
        print(f"❌ Error generating image {i + 1}: {e}\n")


正在生成图片 1 / 4: A Eldritch, Uncanny, Cosmic Horror scene set in 1920s Gothic Horror. The art style is Dark Gothic, Lovecraftian Horror, using Dark, Muted Tones, Sepia, Greenish Black colors. The environment is Foggy coastal town, ancient cyclopean ruins, deep-sea abyss under Stormy night, fog-covered city, eerie full moon. The setting is Providence, Rhode Island, featuring George Gammell Angell, Protagonist. Key events happening in this scene: Death of George Gammell Angell, Mysterious circumstances surrounding the death. The scene is illuminated by Dim gas lamps, eerie green glow, unnatural shadows. Scene description: The narrative begins with the death of the protagonist's grand-uncle, Professor George Gammell Angell, under mysterious circumstances..
图片 1 生成成功，URL: https://oaidalleapiprodscus.blob.core.windows.net/private/org-vlikM8ULjJy9fXt7PwUYao8W/user-KXODt16OC2RhePBBighw3K1K/img-DcLsTFaQzoJnLPIhilY7cy3n.png?st=2025-03-11T02%3A57%3A25Z&se=2025-03-11T04%3A57%3A25Z&sp=r&sv=2024-08-04

## 语音生成

In [ ]:
from pathlib import Path
import openai
import json
from gtts import gTTS  # Google Text-to-Speech (optional alternative)

# ✅ Load the scenes_1.json file
with open("scripts/The_Call_of_Cthulhu/scenes_1.json", "r", encoding="utf-8") as f:
    scene_list = json.load(f)  # Read all scene data

inputs = []  # Store scene text inputs for speech synthesis

# ✅ Iterate through each scene to generate speech
for i, scene in enumerate(scene_list):
    # Extract the original scene text
    input_text = scene["original_text"]

    # ✅ Generate speech using OpenAI TTS
    response = openai.Audio.speech.create(
        model="tts-1",  # Select the text-to-speech model (options: tts-1, tts-1-hd)
        voice="onyx",  # Choose a voice (options: alloy, echo, fable, onyx, nova, shimmer)
        input=input_text  # The text content to be converted into speech
    )

    # ✅ Save the generated audio file
    output_path = f"voice/The_Call_of_Cthulhu/MP3_{i + 1}.mp3"
    response.stream_to_file(output_path)  # Stream the response directly to a file

    print(f"🎵 Audio saved as {output_path}")

# ✅ List available voices in OpenAI TTS
openai.audio.speech.list()


AttributeError: type object 'Audio' has no attribute 'speech'

In [ ]:
import openai

print(dir(openai))  # 查看 OpenAI 模块中所有可用的方法


AttributeError: module 'openai' has no attribute 'openai'

In [ ]:
from gtts import gTTS
import json

# ✅ Load the scenes_1.json file
with open("scripts/The_Call_of_Cthulhu/scenes_1.json", "r", encoding="utf-8") as f:
    scene_list = json.load(f)  # Read all scene data

inputs = []  # Store scene text inputs for speech synthesis

# ✅ Iterate through each scene to generate speech
for i, scene in enumerate(scene_list):
    input_text = scene["original_text"]  # Extract the original scene text

    # ✅ Generate speech using Google Text-to-Speech (gTTS)
    tts = gTTS(text=input_text, lang="en")

    # ✅ Save the generated audio file
    output_path = f"voices/The_Call_of_Cthulhu/MP3_{i + 1}.mp3"
    tts.save(output_path)

    print(f"✅ Audio file generated: {output_path}")


✅ 语音文件已生成: voices/The_Call_of_Cthulhu/MP3_1.mp3
✅ 语音文件已生成: voices/The_Call_of_Cthulhu/MP3_2.mp3
✅ 语音文件已生成: voices/The_Call_of_Cthulhu/MP3_3.mp3
✅ 语音文件已生成: voices/The_Call_of_Cthulhu/MP3_4.mp3


In [32]:
!pip install gtts



In [13]:
!pip install --upgrade --no-cache-dir openai

   ---------------------------------------- 0.0/474.5 kB ? eta -:--:--
   ------------------------ --------------- 286.7/474.5 kB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 474.5/474.5 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [49]:
!pip install moviepy

INFO: pip is looking at multiple versions of moviepy to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/388.3 kB ? eta -:--:--
     ----------------------------------- - 368.6/388.3 kB 11.6 MB/s eta 0:00:01
     -------------------------------------- 388.3/388.3 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/315.4 kB ? eta -:--:--
   --------------------------------------- 315.4/315.4 kB 19.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/22.6 MB ? eta -:--:--
   - -------------------------------------- 1.0/22.6 MB 20.5 MB/s eta 0:00:02
   --- ------------------------------------ 2.2/22.6 MB 28.5 MB/s eta 0:00:01
   ------ --------------------------------- 3.7/22.6 MB 29.2 MB/s eta 0:00:01
   --------- ------------------------------ 5.6/22.6 MB 29.6 MB/s 

!pip show openai

In [ ]:
from moviepy.editor import *
from moviepy.video.io.VideoFileClip import VideoFileClip


# 文件数量（假设你有 1, 2, 3, ... n 个音频 & 图片）
num_files = 4  # 这里可以修改成你的文件数量
video_clips = []  # 存储所有视频片段

for i in range(1, num_files + 1):
    img_file = f"images/The_Call_of_Cthulhu/generated_image_{i}.png"
    audio_file = f"voices/The_Call_of_Cthulhu/MP3_{i}.mp3"
    output_video_file = f"videos/The_Call_of_Cthulhu/temp_video_{i}.mp4"

    # 加载音频
    audio_clip = AudioFileClip(audio_file)
    
    # 加载图片
    # image_clip = ImageClip(img_file).set_duration(AudioFileClip(audio_file).duration)
    image_clip = ImageClip(img_file).set_duration(audio_clip.duration)



    # 绑定音频到图片
    video_clip = image_clip.set_audio(audio_clip)

    # 保存单个视频片段
    video_clip.write_videofile(output_video_file, fps=24, codec="libx264", audio_codec="aac")

    # 添加到视频列表
    video_clips.append(VideoFileClip(output_video_file))

# 拼接所有视频
final_video = concatenate_videoclips(video_clips, method="compose")

# 导出最终合并后的视频
final_video.write_videofile("final_output.mp4", fps=24, codec="libx264", audio_codec="aac")

print("✅ 视频拼接完成: final_output.mp4")


Moviepy - Building video videos/The_Call_of_Cthulhu/temp_video_1.mp4.
MoviePy - Writing audio in temp_video_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos/The_Call_of_Cthulhu/temp_video_1.mp4



TypeError: must be real number, not NoneType

In [ ]:
import os
import subprocess

# ✅ Number of files (Modify this according to your dataset)
num_files = 4  
video_list = []  # Store generated video filenames

# ✅ Ensure the output directory exists
os.makedirs("videos/The_Call_of_Cthulhu", exist_ok=True)

# ✅ Loop through each scene to generate individual videos
for i in range(1, num_files + 1):
    img_file = f"images/The_Call_of_Cthulhu/generated_image_{i}.png"  # Input image file
    audio_file = f"voices/The_Call_of_Cthulhu/MP3_{i}.mp3"  # Input audio file
    output_video_file = f"videos/The_Call_of_Cthulhu/temp_video_{i}.mp4"  # Output video file

    # ✅ Generate a single video using FFmpeg
    ffmpeg_cmd = [
        "ffmpeg", "-loop", "1", "-i", img_file,  # Load image as a still frame
        "-i", audio_file,  # Load audio file
        "-c:v", "libx264", "-tune", "stillimage",  # Encode video using H.264 codec optimized for still images
        "-c:a", "aac", "-b:a", "192k",  # Encode audio using AAC codec with 192kbps bitrate
        "-pix_fmt", "yuv420p",  # Set pixel format for broad compatibility
        "-shortest", output_video_file  # Ensure video duration matches audio length
    ]

    # ✅ Execute FFmpeg command
    subprocess.run(ffmpeg_cmd, check=True)
    video_list.append(f"temp_video_{i}.mp4")  # Store generated video filename

    print(f"✅ Video generated: {output_video_file}")

# ✅ Create a file list for concatenation
concat_file = "videos/The_Call_of_Cthulhu/video_list.txt"
with open(concat_file, "w") as f:
    for video in video_list:
        f.write(f"file '{video}'\n")

# ✅ Concatenate all videos into a final output file
final_output = "Output/final_output.mp4"
ffmpeg_concat_cmd = [
    "ffmpeg", "-f", "concat", "-safe", "0", "-i", concat_file,  # Use FFmpeg to concatenate videos
    "-c", "copy", final_output  # Copy streams without re-encoding
]

subprocess.run(ffmpeg_concat_cmd, check=True)
print(f"✅ Video concatenation complete: {final_output}")


In [ ]:
# 拼接所有视频
final_output = "Output/final_output.mp4"
ffmpeg_concat_cmd = [
    "ffmpeg", "-f", "concat", "-safe", "0", "-i", concat_file,
    "-c", "copy", final_output
]

subprocess.run(ffmpeg_concat_cmd, check=True)
print(f"✅ 视频拼接完成: {final_output}")


✅ 视频拼接完成: final_output.mp4


In [ ]:
!cd videos/The_Call_of_Cthulhu/
!ffmpeg -f concat -safe 0 -i video_list.txt -c copy test_output.mp4
!ls videos/The_Call_of_Cthulhu/video*

/d/TY_1.0/AI/Visual_Novel


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt
  libavutil 

/d/TY_1.0/AI/Visual_Novel


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt
  libavutil 

In [54]:
!pip show moviepy


Name: moviepy
Version: 1.0.3
Summary: Video editing with Python
Home-page: https://zulko.github.io/moviepy/
Author: Zulko 2017
Author-email: 
License: MIT License
Location: c:\users\11091\anaconda3\envs\dl\lib\site-packages
Requires: decorator, imageio, imageio-ffmpeg, numpy, numpy, proglog, requests, tqdm
Required-by: 


In [ ]:
from moviepy.editor import *
from moviepy.editor import VideoFileClip, ImageClip, AudioFileClip, concatenate_videoclips

# 文件数量（假设你有 1, 2, 3, ... n 个音频 & 图片）
num_files = 4  # 这里可以修改成你的文件数量
video_clips = []  # 存储所有视频片段

for i in range(1, num_files + 1):
    img_file = f"images/The_Call_of_Cthulhu/generated_image_{i}.png"
    audio_file = f"voices/The_Call_of_Cthulhu/MP3_{i}.mp3"
    output_video_file = f"videos/The_Call_of_Cthulhu/temp_video_{i}.mp4"

    # 先加载音频（必须在 set_duration 之前加载！）
    audio_clip = AudioFileClip(audio_file)

    # 检查 duration 是否为空
    if audio_clip.duration is None:
        raise ValueError(f"❌ 错误：{audio_file} 没有有效的 duration，请检查音频文件")

    print(f"🎵 音频 {audio_file} 时长: {audio_clip.duration} 秒")

    # 加载图片，并设置持续时间
    image_clip = ImageClip(img_file).set_duration(audio_clip.duration)

    # 绑定音频到图片
    video_clip = image_clip.set_audio(audio_clip)

    # 保存单个视频片段
    video_clip.write_videofile(output_video_file, fps=24, codec="libx264", audio_codec="aac")

    # 添加到视频列表
    video_clips.append(VideoFileClip(output_video_file))

# 拼接所有视频
final_video = concatenate_videoclips(video_clips, method="compose")

# 导出最终合并后的视频
final_video.write_videofile("final_output.mp4", fps=24, codec="libx264", audio_codec="aac")

print("✅ 视频拼接完成: final_output.mp4")


🎵 音频 voices/The_Call_of_Cthulhu/MP3_1.mp3 时长: 35.86 秒
Moviepy - Building video videos/The_Call_of_Cthulhu/temp_video_1.mp4.
MoviePy - Writing audio in temp_video_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos/The_Call_of_Cthulhu/temp_video_1.mp4



TypeError: must be real number, not NoneType